# Academic Journal Summarization Modeling - T5 Model Baseline FineTuning

## Setup

In [ ]:
!pip install -q sentencepiece

In [ ]:
!pip install -q transformers

In [ ]:
!pip install -q evaluate

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.1/84.1 kB 4.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 547.8/547.8 kB 33.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 9.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 64.9/64.9 kB 2.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 6.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 11.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 39.9/39.9 MB 48.4 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
cudf-cu12 24.4.1 requires pyarrow<15.0.0a0,>=14.0.1, but you have pyarrow 17.0.0 which is incompatible.
google-colab 1.0.0 requires requests==2.31.0, but you have requests 2.32.3 which is incompatible.
ibis-framework 8.0.0 requires pyarrow<16,>=2, but you have pyarrow

In [ ]:
!pip install -q rouge_score

  Preparing metadata (setup.py) ... done


In [ ]:
# Standard libraries
import os
import pandas as pd
import numpy as np

# Datset
from datasets import Dataset

# For counting and checkpoints
import pytz
from datetime import datetime

# Modeling
import tensorflow as tf
import nltk
from nltk.tokenize import sent_tokenize

from sklearn.model_selection import train_test_split

from transformers import AutoModelForSeq2SeqLM, AutoTokenizer
from transformers import T5Tokenizer, DataCollatorForSeq2Seq
from transformers import T5ForConditionalGeneration, Seq2SeqTrainingArguments, Seq2SeqTrainer

import evaluate

In [ ]:
# Colab
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

Mounted at /content/drive


## Data

In [ ]:
# df = pd.read_csv('/content/drive/MyDrive/DATASCI266/envisci_articles.csv')
df = pd.read_csv('/content/drive/MyDrive/DATASCI266/envisci_articles_clean.csv')

In [ ]:
# Split data
train_val, test = train_test_split(df, test_size=0.2, random_state=1)
train, val = train_test_split(train_val, test_size=0.25, random_state=1)

In [ ]:
model_name = 't5_finetune_no_prompt'

# Output paths - Baseline
LOCAL_OUTPUT_CHECKPOINT_PATH = f'./drive/MyDrive/DATASCI266/model_checkpoints_t5/{model_name}'
LOCAL_OUTPUT_MODEL_PATH = f'./drive/MyDrive/DATASCI266/model_finetuned_t5_models/{model_name}'

In [ ]:
# Initialize the T5 tokenizer and model

model = AutoModelForSeq2SeqLM.from_pretrained("google/flan-t5-base")
tokenizer = AutoTokenizer.from_pretrained("google/flan-t5-base")

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/1.40k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/990M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/147 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/2.54k [00:00<?, ?B/s]

spiece.model:   0%|          | 0.00/792k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/2.42M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/2.20k [00:00<?, ?B/s]

In [ ]:
def preprocess_function(examples):
  model_inputs = tokenizer(examples['full_text'], max_length = 512, truncation=True)
  labels = tokenizer(examples['abstract'], max_length = 512, truncation = True)
  model_inputs['labels'] = labels['input_ids']
  return model_inputs

tokenized_datasets_train = train.apply(lambda row: preprocess_function(row), axis=1)
tokenized_datasets_val = val.apply(lambda row: preprocess_function(row), axis=1)

In [ ]:
tokenized_datasets_train = tokenized_datasets_train.to_list()
dataset_train = Dataset.from_list(tokenized_datasets_train)

tokenized_datasets_val = tokenized_datasets_val.to_list()
dataset_val = Dataset.from_list(tokenized_datasets_val)

In [ ]:
# https://huggingface.co/learn/nlp-course/chapter7/5?fw=pt#summarization

import numpy as np
import evaluate
import nltk
nltk.download('punkt')

rouge_score = evaluate.load("rouge")

def compute_metrics(eval_pred):
    predictions, labels = eval_pred
    # Decode generated summaries into text
    decoded_preds = tokenizer.batch_decode(predictions, skip_special_tokens=True)
    # Replace -100 in the labels as we can't decode them
    labels = np.where(labels != -100, labels, tokenizer.pad_token_id)
    # Decode reference summaries into text
    decoded_labels = tokenizer.batch_decode(labels, skip_special_tokens=True)
    # ROUGE expects a newline after each sentence
    decoded_preds = ["\n".join(sent_tokenize(pred.strip())) for pred in decoded_preds]
    decoded_labels = ["\n".join(sent_tokenize(label.strip())) for label in decoded_labels]
    # Compute ROUGE scores
    result = rouge_score.compute(
        predictions=decoded_preds, references=decoded_labels, use_stemmer=True
    )
    # Extract the median scores
    # result = {key: value.mid.fmeasure * 100 for key, value in result.items()}
    return {k: round(v, 4) for k, v in result.items()}

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


In [ ]:
from transformers import Seq2SeqTrainer
data_collator = DataCollatorForSeq2Seq(tokenizer, model=model)
# Fine Tuning
# Params
learning_rate = 0.00005
batch_size = 4
per_device_eval_batch = 4
weight_decay = 0.01
save_total_limit = 3
num_epochs = 5
# logging_steps = len(lst_texts) // batch_size

# Set up training arguments
training_args = Seq2SeqTrainingArguments(
   output_dir=LOCAL_OUTPUT_CHECKPOINT_PATH,
   evaluation_strategy="epoch",
   learning_rate=learning_rate,
   per_device_train_batch_size=batch_size,
   per_device_eval_batch_size=per_device_eval_batch,
   weight_decay=weight_decay,
   save_total_limit=save_total_limit,
   num_train_epochs=num_epochs,
   predict_with_generate=True,
  #  logging_steps=logging_steps,
   push_to_hub=False
)

trainer = Seq2SeqTrainer(
    model,
    training_args,
    train_dataset=dataset_train,
    eval_dataset=dataset_val,
    data_collator=data_collator,
    tokenizer=tokenizer,
    compute_metrics=compute_metrics
)

trainer.train()
trainer.save_model(LOCAL_OUTPUT_MODEL_PATH)

/usr/local/lib/python3.10/dist-packages/transformers/training_args.py:1494: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


Epoch,Training Loss,Validation Loss,Rouge1,Rouge2,Rougel,Rougelsum
1,3.053600,2.758755,0.088600,0.027600,0.071900,0.077200
2,2.963900,2.740196,0.089100,0.028400,0.072500,0.077600
3,2.878000,2.726604,0.089100,0.028800,0.072500,0.078000
4,2.883600,2.721205,0.088300,0.028500,0.071800,0.077000
5,2.835800,2.720796,0.087500,0.028400,0.071200,0.076400


/usr/local/lib/python3.10/dist-packages/transformers/generation/utils.py:1249: UserWarning: Using the model-agnostic default `max_length` (=20) to control the generation length. We recommend setting `max_new_tokens` to control the maximum length of the generation.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/transformers/generation/utils.py:1249: UserWarning: Using the model-agnostic default `max_length` (=20) to control the generation length. We recommend setting `max_new_tokens` to control the maximum length of the generation.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/transformers/generation/utils.py:1249: UserWarning: Using the model-agnostic default `max_length` (=20) to control the generation length. We recommend setting `max_new_tokens` to control the maximum length of the generation.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/transformers/generation/utils.py:1249: UserWarning: Using the model-agnostic default `max_length` (=20) to control